# EV Charging Station Siting Analysis
### Applied Data Science Capstone by IBM/Coursera
### Author: Rubenka Bandyopadhyay

## Table of contents
* [Introduction: Business Problem Outline and Target Audience](#introduction)
* [Data Sources](#data)
* [Methodology](#methodology)
* [Results](#results)
* [Discussion](#discussion)
* [Conclusion](#conclusion)

## Introduction: Business Problem Outline and Target Audience <a name="introduction"></a>

_Background:_ This code was written as part of the requirements of the IBM/Coursera Data Science Professional Career Track Specialization. Students were asked to choose a problem and utilize skills acquired during this specialization to solve the selected problem.<br>

_Business Problem Outline_ <br>

Electric Vehicles (EVs) are gaining popularity due to absence of local emissions, and certain technical advantages that EVs have over conventional vehicles (faster acceleration, no need for periodic service requirements etc.). This is not to say that there are no barriers to the widespread acceptance of EVs. One of the primary drawbacks is the lack of access to charging equipment at public locations.

In this project, we look at the city of Raleigh in North Carolina, where I currently reside. The objective is to try to come up with a siting analysis for public EV charging station installations - preferably close to local shops and restaurants so that people can charge their cars while getting groceries or sharing a meal with their loved ones. One of the indirect benefits of EV charging that is often talked about is contribution to the local economy - i.e. people tend to spend money on nearby shops when waiting on charging their EVs.

To be specific, the area of interest in this case is within a 100 km radius of downtown Raleigh in North Carolina. This approximately covers the towns of Durham, Raleigh and Chapel Hill. As of 2020, the population of the larger Raleigh-Durham-Chapel Hill Combined Statistical Area (CSA) is estimated at 2.03 million. Hailed as a technological hub within the state, the mean age of residents range between 26 to 35 years. This area may be considered a suitable case study location for testing out algorithms ranking retail facilities that could benefit from EV charger installations.

Note that the approach that I am taking is pretty simplistic - I am aiming to identify retail locations (primarily grocery shops, restaurants and some service industries) that have relatively few EV chargers in the vicinity. A list of suitable retail locations that could benefit from an EV charging installation in its neighborhood is provided as input to a clustering algorithm. This algorithm determines locations that could improve access to EV charging in our region of interest.
Outputs include a map with clusters of retail locations that could benefit from EV charger installations and a list of suitable locations for EV charging installations.


_Target Audience_ <br> 

This project is primarily aimed at any one interested in Data Science applications in the Energy sector, but I hope that other Data Science enthusiasts will also find ways to repurpose the code. Data Science techniques and/or Machine Learning Algorithms often continue to remain abstract ideas until we apply them to problems. I hope this project provides some insight into translating abstract ideas into specific outputs that could aid decision making in business ventures. 

## Data Sources <a name="data"></a>

The following publicly available data-sets have been used to perform the analysis
1. <a href = "https://foursquare.com/"> API Foursquare - This database provides information on locations of popular restaurants </a>
2. <a href = "https://afdc.energy.gov/fuels/electricity_locations.html#/find/nearest?fuel=ELEC"> AFDC Database - This database provides information on preexisting EV charging locations </a>


## Methodology <a name="methodology"></a>

__Step 0__: Import relevant Python libraries

In [1]:
import pandas as pd # this library offers data structures and operations for manipulating numerical tables 
import numpy as np # this library is used for Scientific Computing

import requests # library to handle requests
import random # library for random number generation

# Following line needs to be uncommented if geopy is not installed
#!conda install -c conda-forge geopy --yes 

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

# Import libraries for clustering analysis
import sklearn.neighbors
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# Following line needs to be uncommented if geopy is not installed
#!conda install -c conda-forge folium=0.5.0 --yes

import folium # plotting library

print('Libraries imported.')

Libraries imported.


__Step 1__: Read in the data from AFDC and Foursquare API

In [2]:
# Importing locations of existing public EVSE chargers from AFDC
EVSE = pd.read_excel(open('AFDC_Data.xlsx','rb'), sheet_name='AFDC_Master_Data_NC')
EVSE.head()

,Station Name,Street Address,City,ZIP,EV Level1 EVSE Num,EV Level2 EVSE Num,EV DC Fast Count,Latitude,Longitude,Date Last Confirmed,ID
0,DUKE ENERGY,410 S Mint St,Charlotte,28202,NaN,1.0,NaN,35.226914,-80.850182,2020-05-17,38892
1,City of Raleigh - Municipal Building,285 W Hargett St,Raleigh,27601,NaN,2.0,NaN,35.778416,-78.643470,2019-11-08,39016
2,City of Raleigh - Downtown,215 W Cabarrus St,Raleigh,27601,NaN,1.0,NaN,35.774350,-78.642287,2019-11-08,39017
3,Modern Nissan - Concord,967 Concord Pkwy S,Concord,28027,NaN,1.0,1.0,35.392063,-80.622777,2019-09-09,40066
4,Fred Anderson Nissan,4559 Raeford Rd,Fayetteville,28304,NaN,1.0,1.0,35.042419,-78.956747,2019-09-09,40067


In [3]:
# Use Foursquare API to download places of interest within the RDU-Chapel Hill Area

Following cell contains foursquare API credentials and has been removed from public view

In [5]:
# Testing connection with Foursquare API

address = '4242 Six Forks Rd, Raleigh, NC 27609'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

35.83801365 -78.64074000129678


Next up - identification of trending spots in the Raleigh... <br> There are a few different ways of identifying trending spots. For purposes of this project we consider two specific Foursquare categories:<br> 
(a) 'Food', and (b) 'Shops & Services'. <br> Refer to <a href = "https://developer.foursquare.com/docs/build-with-foursquare/categories/"> API Foursquare List of Venue Categories </a> for full list of categories

URI to search for a specific venue category
> `https://api.foursquare.com/v2/venues/`**search**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&query=`**QUERY**`&radius=`**RADIUS**`&limit=`**LIMIT**

For reference, here is the <a href = "https://developer.foursquare.com/docs/build-with-foursquare/categories/"> link </a> to Foursquare API's webpage that lists all parameter definitions for URI 

In [6]:
#
search_query = 'Shops & Services'
radius = 100000 # radius in meters
LIMIt = 500 # Limit on number of venues
print(search_query + ' .... OK!')

Shops & Services .... OK!


Uncomment the following section for execution after adding client_id and client name 

In [2]:
#url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

In [8]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ed44da40de0d9001bbff517'},
 'response': {'venues': [{'id': '4ad4c00af964a52037ed20e3',
    'name': 'Shops of Cameron Village',
    'location': {'address': '2034 Cameron St',
     'crossStreet': 'at Daniels St',
     'lat': 35.791233,
     'lng': -78.660793,
     'labeledLatLngs': [{'label': 'display',
       'lat': 35.791233,
       'lng': -78.660793}],
     'distance': 5513,
     'postalCode': '27605',
     'cc': 'US',
     'city': 'Raleigh',
     'state': 'NC',
     'country': 'United States',
     'formattedAddress': ['2034 Cameron St (at Daniels St)',
      'Raleigh, NC 27605',
      'United States']},
    'categories': [{'id': '5744ccdfe4b0c0459246b4dc',
      'name': 'Shopping Plaza',
      'pluralName': 'Shopping Plazas',
      'shortName': 'Shopping Plaza',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/mall_',
       'suffix': '.png'},
      'primary': True}],
    'venuePage': {'id': '61850785'},
    'referralId': '

In [9]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
Retail_df = json_normalize(venues)
Retail_df.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id
0,4ad4c00af964a52037ed20e3,Shops of Cameron Village,"[{'id': '5744ccdfe4b0c0459246b4dc', 'name': 'S...",v-1590971835,False,2034 Cameron St,at Daniels St,35.791233,-78.660793,"[{'label': 'display', 'lat': 35.791233, 'lng':...",5513,27605,US,Raleigh,NC,United States,"[2034 Cameron St (at Daniels St), Raleigh, NC ...",61850785
1,4ddbfc573151ee0807532b88,Shops of Cameron Village CAT Bus Stop,"[{'id': '4bf58dd8d48988d1fe931735', 'name': 'B...",v-1590971835,False,Cameron at Daniels,NaN,35.791208,-78.660974,"[{'label': 'display', 'lat': 35.79120760073099...",5521,NaN,US,Raleigh,NC,United States,"[Cameron at Daniels, Raleigh, NC, United States]",NaN
2,4b5eebaff964a520f79d29e3,Park Shops,"[{'id': '4bf58dd8d48988d198941735', 'name': 'C...",v-1590971835,False,101 Current Dr,at North Carolina State University,35.785491,-78.667355,"[{'label': 'display', 'lat': 35.7854910218201,...",6321,27695,US,Raleigh,NC,United States,[101 Current Dr (at North Carolina State Unive...,NaN
3,5c3fcfaf35811b002cdeccf1,Shops At Mcneil Pointe,"[{'id': '5744ccdfe4b0c0459246b4dc', 'name': 'S...",v-1590971835,False,NaN,NaN,35.812685,-78.627096,"[{'label': 'display', 'lat': 35.81268494095342...",3076,27608,US,Raleigh,NC,United States,"[Raleigh, NC 27608, United States]",NaN
4,52c81e8811d2d004d457e8ba,Bass Pro Shops,"[{'id': '4bf58dd8d48988d1f2941735', 'name': 'S...",v-1590971835,False,801 Bass Pro Ln,N Harrison Ave,35.830310,-78.770704,"[{'label': 'display', 'lat': 35.8303096, 'lng'...",11760,27513,US,Cary,NC,United States,"[801 Bass Pro Ln (N Harrison Ave), Cary, NC 27...",NaN


In [10]:
# Tracking down restaurants in the area
search_query = 'Food'
radius = 100000 # radius in meters
LIMIt = 500 # Limit on number of venues
print(search_query + ' .... OK!')

Food .... OK!


In [11]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=3LV40XKAW4DLMFQYMKV4V2EU0L1TCFVLCQ5PMVIGX44YT4GP&client_secret=ES2DAIMEHEO4KSY2YENFMUHCPMOVDLXPA1PVME3XQZQATFIB&ll=35.83801365,-78.64074000129678&v=20180604&query=Food&radius=100000&limit=500'

In [12]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ed44c36006dce001b983f0d'},
 'response': {'venues': [{'id': '4b22b0e6f964a520e04b24e3',
    'name': 'Food Lion Grocery Store',
    'location': {'address': '5426 Six Forks Rd',
     'lat': 35.854909950587164,
     'lng': -78.64072713192202,
     'labeledLatLngs': [{'label': 'display',
       'lat': 35.854909950587164,
       'lng': -78.64072713192202}],
     'distance': 1880,
     'postalCode': '27609',
     'cc': 'US',
     'city': 'Raleigh',
     'state': 'NC',
     'country': 'United States',
     'formattedAddress': ['5426 Six Forks Rd',
      'Raleigh, NC 27609',
      'United States']},
    'categories': [{'id': '52f2ab2ebcbc57f1066b8b46',
      'name': 'Supermarket',
      'pluralName': 'Supermarkets',
      'shortName': 'Supermarket',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1590971835',
    'hasPerk': False},
   {'id': '4ed01

In [13]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
Restaurants_Groceries_df = json_normalize(venues)
Restaurants_Groceries_df.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,...,location.formattedAddress,location.crossStreet,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,venuePage.id,location.neighborhood
0,4b22b0e6f964a520e04b24e3,Food Lion Grocery Store,"[{'id': '52f2ab2ebcbc57f1066b8b46', 'name': 'S...",v-1590971835,False,5426 Six Forks Rd,35.854910,-78.640727,"[{'label': 'display', 'lat': 35.85490995058716...",1880,...,"[5426 Six Forks Rd, Raleigh, NC 27609, United ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4ed01e2b775bbb5f32900c8a,Target - Food Avenue,"[{'id': '4bf58dd8d48988d120951735', 'name': 'F...",v-1590971835,False,North Hills,35.838319,-78.641821,"[{'label': 'display', 'lat': 35.8383193183181,...",103,...,"[North Hills, Raleigh, NC 27609, United States]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4b071d63f964a52087f722e3,Food Lion Grocery Store,"[{'id': '52f2ab2ebcbc57f1066b8b46', 'name': 'S...",v-1590971835,False,4317 Fall Of The Neuse Rd,35.841043,-78.613129,"[{'label': 'display', 'lat': 35.84104260746239...",2514,...,"[4317 Fall Of The Neuse Rd, Raleigh, NC 27609,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4b8ad3d9f964a520c38332e3,Food Lion Grocery Store,"[{'id': '52f2ab2ebcbc57f1066b8b46', 'name': 'S...",v-1590971835,False,5633 Creedmoor Rd,35.858978,-78.682036,"[{'label': 'display', 'lat': 35.8589780724059,...",4396,...,"[5633 Creedmoor Rd, Raleigh, NC 27612, United ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5a737f28d41bb73f64b3875e,Transfer Co. Food Hall,"[{'id': '4bf58dd8d48988d120951735', 'name': 'F...",v-1590971835,False,500 E Davie St,35.775049,-78.632151,"[{'label': 'display', 'lat': 35.77504928706339...",7051,...,"[500 E Davie St (S. East Street), Raleigh, NC ...",S. East Street,1902294,https://www.grubhub.com/restaurant/transfer-co...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,541552628,NaN


Now that we have a list of retail services, groceries and restaurants in the area, let's combine the two data-sets 

In [14]:
Retail_df = Retail_df[['name', 'location.lat', 'location.lng']]

In [15]:
Restaurants_Groceries_df = Restaurants_Groceries_df[['name', 'location.lat', 'location.lng']]

In [16]:
Locations_Venue = pd.concat([Retail_df, Restaurants_Groceries_df], ignore_index=True)
Locations_Venue.rename(columns={'name':'Venue Name'}, 
                 inplace=True)
Locations_Venue.head()

,Venue Name,location.lat,location.lng
0,Shops of Cameron Village,35.791233,-78.660793
1,Shops of Cameron Village CAT Bus Stop,35.791208,-78.660974
2,Park Shops,35.785491,-78.667355
3,Shops At Mcneil Pointe,35.812685,-78.627096
4,Bass Pro Shops,35.830310,-78.770704


__Step 2:__ Now that we have the geographic coordinates of businesses in the area, let us determine the distance between each venue obtained from the foursquare API and each EVSE outlet location: <br>
Procedure to calculate the distance has been obtained from this article in medium: <a href = "https://medium.com/@danalindquist/finding-the-distance-between-two-lists-of-geographic-coordinates-9ace7e43bb2f"> Finding the distance between two lists of geographic coordinates </a> by Dana Lindquist

In [17]:
# Create two dataframes with names of EVSE outlets and retail venues, and lat-long in degrees

Locations_EVSE = EVSE[['Station Name','Latitude', 'Longitude']].copy()

Locations_EVSE.rename(columns={'Station Name':'EVSE Station Name'}, 
                 inplace=True)


In [18]:
# add columns with radians for latitude and longitude
Locations_EVSE[['Latitude', 'Longitude']] = (
    np.radians(Locations_EVSE.loc[:,['Latitude', 'Longitude']])
)
Locations_Venue[['location.lat', 'location.lng']] = (
    np.radians(Locations_Venue.loc[:,['location.lat', 'location.lng']])
)

The distance computed here is a haversine distance. This assumes the earth is a true sphere which makes for a relatively fast computation. The sklearn computation assumes the radius of the sphere is 1, so to get the distance in miles we multiply the output of the sklearn computation by 3959 miles, the average radius of the earth. To get the distance in kilometers this number would be 6371 km.

In [19]:
dist = sklearn.neighbors.DistanceMetric.get_metric('haversine')
dist_matrix = (dist.pairwise
    (Locations_EVSE[['Latitude', 'Longitude']],
     Locations_Venue[['location.lat', 'location.lng']])*6371
)
# Note that 6371 is the radius of the earth in km
df_dist_matrix = (
    pd.DataFrame(dist_matrix,index=Locations_EVSE['EVSE Station Name'], 
                 columns=Locations_Venue['Venue Name'])
)

In [20]:
df_dist_matrix.head()

Venue Name,Shops of Cameron Village,Shops of Cameron Village CAT Bus Stop,Park Shops,Shops At Mcneil Pointe,Bass Pro Shops,Shops at Brennan Station,Shops at Town Station,Shops at Oberlin Court,Flip Flop Shops,Shops,...,Food Lion Grocery Store,Food Lion Grocery Store,Food Lion Grocery Store,Food Lion Grocery Store,Food Lion Grocery Store,Food Lion Grocery Store,Food Lion Grocery Store,Food Lion Grocery Store,Food Lion,Food Lion Grocery Store
EVSE Station Name,,,,,,,,,,,,,,,,,,,,,
DUKE ENERGY,207.864559,207.848091,207.112806,211.470816,199.776821,209.861712,194.649437,208.165049,207.744806,196.849571,...,226.209711,190.425256,191.718493,198.938751,188.640255,187.869197,222.376446,192.217080,194.081965,193.306387
City of Raleigh - Municipal Building,2.114912,2.125133,2.293743,4.086690,12.843331,13.894064,14.743878,2.628716,7.738003,12.135436,...,22.262171,18.532694,26.582474,30.475037,21.865326,32.346114,23.468931,24.119136,37.038536,34.598061
City of Raleigh - Downtown,2.512168,2.520966,2.578477,4.477435,13.146853,14.358494,14.834826,3.067138,8.191014,12.151892,...,21.979695,18.523877,26.904986,30.892105,21.682556,32.663094,23.091992,23.782771,37.435188,34.982602
Modern Nissan - Concord,182.871199,182.854637,182.147432,186.388969,174.371684,184.191349,169.568757,183.129647,182.411820,171.932385,...,202.169735,165.549228,165.707766,172.543602,164.348926,161.643572,198.707459,168.589199,167.462919,166.810293
Fred Anderson Nissan,87.476766,87.469043,86.687799,90.708159,89.215991,98.474145,82.219775,88.215571,92.196897,80.933342,...,87.377717,77.699954,95.379101,105.666144,69.517079,97.695319,80.791157,63.085343,108.085778,105.153412


__Step 3:__ Now let's count the number of EVSE Stations that are at distance which is less than or equal to 5 km from the prospective venues

In [21]:
# Defining an indicator matrix

df_indicator_matrix = df_dist_matrix.le(5).astype(int)
df_indicator_matrix.shape

(640, 100)

In [22]:
# Obtain the sum of each venue (column) in the indicator matrix - this gives the total number of EVSEs within 5 km of the said venue

Num_EVSE = df_indicator_matrix.sum(axis = 0)
Num_EVSE.head(30)

Venue Name
Shops of Cameron Village                       34
Shops of Cameron Village CAT Bus Stop          34
Park Shops                                     34
Shops At Mcneil Pointe                         30
Bass Pro Shops                                 11
Shops at Brennan Station                        5
Shops at Town Station                          11
Shops at Oberlin Court                         42
Flip Flop Shops                                13
Shops                                          12
shops of Lafayette Village                      5
Park Shops 130                                 34
Suzio's Boutique (at Shops of Baileywick)       5
Shopsmith Repair/Woodworking Academy           15
Rainbow Shops                                   4
Rainbow Shops                                   4
Park Shops 201                                 34
Fun and Fabulous (at Shops of Baileywick)       5
Fancy That by A&E (at Shops of Baileywick)      5
Target                                 

In [23]:
# Select venues that have less than 2 EVSE outlets within 5 kms
venue_names = Num_EVSE[(Num_EVSE <= 2)]

# Use previously selected venue names and identify the corresponding coordinates, 
# based on dataframe previously geneerated from Foursquare API data 
Venue_Selected = Locations_Venue.loc[Locations_Venue['Venue Name'].isin(venue_names.index)]
# Also convert the latitudes and longitudes back to degrees from radians
Venue_Selected[['location.lat', 'location.lng']] = (
    np.degrees(Locations_Venue.loc[:,['location.lat', 'location.lng']])
)

Venue_Selected.head()


C:\Users\Rubenka\Anaconda3\lib\site-packages\pandas\core\frame.py:3494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,Venue Name,location.lat,location.lng
14,Rainbow Shops,35.873154,-78.582456
15,Rainbow Shops,35.798513,-78.579753
21,The Shops at Timber Landing shopping center,35.684866,-78.616349
22,The Shops at Garner Plaza,35.704787,-78.611581
26,Rainbow Shops,35.798753,-78.507796


__Step 4:__ Let us now __cluster__ those locations to create centers of zones containing good locations. Those zones, their centers and addresses will be the final result of our analysis.

In [24]:
number_of_clusters = 7
Venue_Selected_xys = Venue_Selected[['location.lat', 'location.lng']].values
kmeans = KMeans(n_clusters=number_of_clusters, random_state=0).fit(Venue_Selected_xys)

cluster_centers = kmeans.cluster_centers_

print(cluster_centers)
print(kmeans.labels_)

[[ 35.58894232 -78.78396421]
 [ 35.84670328 -78.58942496]
 [ 35.9688596  -78.88098186]
 [ 35.76826896 -78.79247378]
 [ 35.6908554  -78.61028009]
 [ 35.89810002 -78.69217356]
 [ 35.8258226  -78.44740955]]
[1 1 4 4 6 4 4 5 6 0 5 4 2 2 0 1 1 5 2 5 1 1 1 3 1 5 1 5 1 5 1 5 1 6 4 4 3
 4 3 6 3 3 4 2 3 3 6 4 6 3 2 2 0 2 4 0 2]


## Results <a name="results"></a>

In [25]:
# add clustering labels

Venue_labels = Venue_Selected

Venue_labels.insert(0, 'Cluster Labels', kmeans.labels_)



In [26]:
Venue_labels.head()

,Cluster Labels,Venue Name,location.lat,location.lng
14,1,Rainbow Shops,35.873154,-78.582456
15,1,Rainbow Shops,35.798513,-78.579753
21,4,The Shops at Timber Landing shopping center,35.684866,-78.616349
22,4,The Shops at Garner Plaza,35.704787,-78.611581
26,6,Rainbow Shops,35.798753,-78.507796


In [27]:
# create map

map_clusters = folium.Map(location=[35.78, -78.64], zoom_start=11)

# set color scheme for the clusters
x = np.arange(number_of_clusters)
ys = [i + x + (i*x)**2 for i in range(number_of_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Venue_labels['location.lat'], Venue_labels['location.lng'], Venue_labels['Venue Name'], Venue_labels['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Finally, let's reverse geocode those k-means clustering centers to get the addresses which can be presented to stakeholders.

In [28]:
geolocator = Nominatim(user_agent="foursquare_agent")

lat = 35.74137968
long = -78.59448015

for lat, long in cluster_centers:
    location = geolocator.reverse([lat, long])
    print(location.address, "\n")

Quail Hill Street, Five Points, Fuquay-Varina, Wake County, North Carolina, 27526, United States of America 

9475, Millbrook Woods Drive, Millbrook Green Apartments, Mini City, Raleigh, Wake County, North Carolina, 27604, United States of America 

1700, Cooper Street, Hayti, Hayes, Durham, Durham County, North Carolina, 27703, United States of America 

199, Arbuckle Lane, Kildaire Farms, Cary, Wake County, North Carolina, 27511, United States of America 

505, Southerby Drive, Southview, Garner, Wake County, North Carolina, 27529, United States of America 

3378, Brennan Drive, Brandon Station, Lake Lynn, Raleigh, Wake County, North Carolina, 27613, United States of America 

Marks Creek Road, Riley Hill, Wake County, North Carolina, 27545, United States of America 



## Discussion <a name="discussion"></a>

While this project does a good job demonstrating how data science can be used to inform energy infrastructure siting decisions, the analysis could be improved in many different ways. A few are listed here but should not be considered an exhaustive list:<br> <br> (a) Socio-economic as well as demographic factors need to be considered when prioritizing siting locations. Here is an <a href = "https://theicct.org/sites/default/files/publications/Expanding-access-electric-mobility_ICCT-Briefing_06122017_vF.pdf"> excellent article </a> summarizing policies and actions being taken to expand access to electric transporation among low income groups and apartment dwellers <br> <br>
(b) Siting analysis may need to take into considerations the practicality of EV charging installations - zoning permits, transmission capacity to support supply of electricity for EV charging etc. are some factors that could affect EV charging installations <br> <br>
(c) Demand for EV charging is another really important factor. If there are not enough EVs driving through the region that need to stop and charge - there may not be any incentive for installation of EV chargers. North Carolina DMV has recently started releasing <a href = "https://www.ncdot.gov/initiatives-policies/environmental/climate-change/Pages/zev-registration-data.aspx"> EV registration data in NC counties </a>. This could indicate how many EV owners currently reside in NC. Market research could also be performed to explore the out-of-state EV traffic passing through NC. 


## Conclusion <a name="conclusion"></a>

This project provides a simple example of utilization of k-means clustering technique in the clean energy sector. Data cleaning and manipulation, application of algorithms, and subsequent data visualization are the primary steps involved in any problem that needs to be solved using data analytics. I hope that this notebook provides a source of reference for those starting to explore data-driven solutions to their business problems.